# Test Stripe Financial Functions

Test all three core functions:
1. **Get Accounts** - Retrieve balances
2. **Get Transactions** - Monitor spending
3. **Transfer Funds** - Move money between accounts

## Setup

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../secrets.env")
load_dotenv("../variables.env")

# Add parent directory to path
sys.path.append('..')

from stripe_integration.client import StripeFinancialClient

# Get session ID
STRIPE_SESSION_ID = os.getenv("STRIPE_SESSION_ID")

if not STRIPE_SESSION_ID:
    print("❌ STRIPE_SESSION_ID not found in secrets.env")
    print("   Add it after running stripe_setup_clean.ipynb")
else:
    print(f"✅ Session ID loaded: {STRIPE_SESSION_ID[:30]}...")
    
# Create client
client = StripeFinancialClient()
print("✅ Stripe client initialized")

## Test 1: Get Accounts & Balances

Retrieve all connected accounts with their current balances.

In [ ]:
print("="*60)
print("TEST 1: Get Accounts")
print("="*60)

accounts = await client.get_accounts(STRIPE_SESSION_ID)

if accounts:
    print(f"\n✅ Found {len(accounts)} accounts:\n")
    
    total = 0
    for idx, acc in enumerate(accounts, 1):
        print(f"{idx}. 💰 {acc.name}")
        print(f"   Balance: ${acc.balance:,.2f}")
        print(f"   Type: {acc.type.title()}")
        print(f"   Bank: {acc.institution_name}")
        print(f"   Last 4: {acc.last_four}")
        print(f"   ID: {acc.id}")
        print()
        total += acc.balance
    
    print("="*60)
    print(f"Total Balance: ${total:,.2f}")
    print("="*60)
else:
    print("\n❌ No accounts found")
    print("   Make sure you've connected accounts via stripe_setup_clean.ipynb")

## Test 2: Get Transactions

Retrieve and categorize recent transactions.

In [ ]:
print("="*60)
print("TEST 2: Get Transactions")
print("="*60)

if not accounts:
    print("\n⚠️  Run Test 1 first to load accounts")
else:
    # Get transactions for first account
    test_account = accounts[0]
    print(f"\nFetching transactions for: {test_account.name}\n")
    
    transactions = await client.get_transactions(test_account.id, days=30)
    
    if transactions:
        print(f"✅ Found {len(transactions)} transactions\n")
        print("-" * 60)
        
        # Group by category
        by_category = {}
        for txn in transactions:
            if txn.category not in by_category:
                by_category[txn.category] = []
            by_category[txn.category].append(txn)
        
        # Show by category
        for category in sorted(by_category.keys()):
            txns = by_category[category]
            total = sum(txn.amount for txn in txns)
            
            print(f"\n📊 {category}: ${abs(total):,.2f} ({len(txns)} transactions)")
            
            # Show first 3 transactions
            for txn in txns[:3]:
                sign = "+" if txn.amount > 0 else ""
                date_str = txn.date.strftime("%b %d")
                print(f"   {date_str}: {sign}${txn.amount:,.2f} - {txn.description}")
            
            if len(txns) > 3:
                print(f"   ... and {len(txns) - 3} more")
        
        # Summary
        total_spent = sum(abs(txn.amount) for txn in transactions if txn.amount < 0)
        total_income = sum(txn.amount for txn in transactions if txn.amount > 0)
        
        print("\n" + "="*60)
        print(f"Total Spent: ${total_spent:,.2f}")
        print(f"Total Income: ${total_income:,.2f}")
        print(f"Net: ${total_income - total_spent:,.2f}")
        print("="*60)
        
    else:
        print("ℹ️  No transactions found for this account")
        print("   (Test accounts may not have transaction data)")

## Test 3: Transfer Funds

Simulate transferring money between accounts.

In [ ]:
print("="*60)
print("TEST 3: Transfer Funds")
print("="*60)

if not accounts or len(accounts) < 2:
    print("\n⚠️  Need at least 2 accounts to test transfers")
    print("   Connect more accounts via stripe_setup_clean.ipynb")
else:
    # Test transfer from first to second account
    from_account = accounts[0]
    to_account = accounts[1]
    transfer_amount = 100.00
    
    print(f"\nTransfer Details:")
    print(f"  From: {from_account.name} (${from_account.balance:,.2f})")
    print(f"  To: {to_account.name} (${to_account.balance:,.2f})")
    print(f"  Amount: ${transfer_amount:,.2f}")
    print()
    
    # Check if sufficient funds
    if from_account.balance < transfer_amount:
        print(f"❌ Insufficient funds!")
        print(f"   {from_account.name} only has ${from_account.balance:,.2f}")
        print(f"   Cannot transfer ${transfer_amount:,.2f}")
    else:
        print("Attempting transfer...\n")
        
        success = await client.transfer_funds(
            from_account.id,
            to_account.id,
            transfer_amount,
            "Test transfer"
        )
        
        if success:
            print("✅ Transfer successful!")
            print()
            print("Note: In test mode, this is a simulated transfer.")
            print("Balances shown above won't actually change.")
            print()
            print("After transfer would be:")
            print(f"  {from_account.name}: ${from_account.balance - transfer_amount:,.2f}")
            print(f"  {to_account.name}: ${to_account.balance + transfer_amount:,.2f}")
        else:
            print("❌ Transfer failed")
            print("   Check logs for details")

## Test 4: Get Single Account Balance

Test the get_balance() method for a specific account.

In [ ]:
print("="*60)
print("TEST 4: Get Single Account Balance")
print("="*60)

if not accounts:
    print("\n⚠️  Run Test 1 first to load accounts")
else:
    test_account = accounts[0]
    print(f"\nChecking balance for: {test_account.name}\n")
    
    balance = await client.get_balance(test_account.id)
    
    print(f"✅ Current balance: ${balance:,.2f}")
    print(f"   Account: {test_account.name}")
    print(f"   ID: {test_account.id}")

## Summary

All tests complete! The Stripe Financial integration is working:

✅ **Get Accounts** - Retrieves all connected accounts with balances  
✅ **Get Transactions** - Fetches and categorizes transaction history  
✅ **Transfer Funds** - Validates and simulates transfers  
✅ **Get Balance** - Retrieves individual account balances  

**Next Steps:**
1. These functions are ready to use in the Money Manager agent
2. Can be wrapped in FastAPI endpoints for API access
3. Ready for Budget Helper agent integration